In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from matplotlib import pyplot as plt


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Reading dataset and saving into a dataframe

In [2]:
df=pd.read_csv("/kaggle/input/robi-datathon-2-pre-assessment/train.csv", )

In [3]:
df.head()

In [4]:
df.shape

# Check number of null values in columns

In [5]:
df.isna().sum()

# Droping columns with null values

In [6]:
df2=df.dropna(axis='columns')

In [7]:
df2.shape

In [8]:
df2.info

In [9]:
df2.shape

# Manually Checking Columns with Null Value and Dropping Columns

In [10]:
df2.value_counts("s53")

In [11]:
df2 = df2.drop(columns = ['s53'])

In [12]:
df2.info()

In [13]:
df2.value_counts()

In [14]:
df2.describe()

In [15]:

df2.value_counts("s11")


In [16]:
df2.value_counts("s12")

In [17]:
df2.value_counts("s13")

In [18]:
df2.value_counts("s16")

In [19]:
df2.value_counts("s17")

In [20]:
df2.value_counts("s18")

In [21]:
df2.value_counts("s11")

# Replacing String Values with Numerical values

In [22]:
df2['gender'].replace({'M':1, 'F':0}, inplace = True)
df2['s11'].replace({'N':0, 'Y':1}, inplace = True)
df2['s12'].replace({'N':0, 'Y':1}, inplace = True)
df2['s16'].replace({'A':0, 'B':1, 'C':2, 'D':3}, inplace = True)
df2['s17'].replace({'A':0, 'B':1, 'C':2, 'D':3}, inplace = True)
df2['s18'].replace({'A':0, 'B':1, 'C':2, 'D':3}, inplace = True)
df2['s58'].replace({'A':0, 'B':1, 'C':2, 'D':3}, inplace = True)
df2['s70'].replace({'op: A':0, 'op: B':1, 'op: C':2, 'op: D':3}, inplace = True)
df2['s71'].replace({'a':0, 'b':1, 'c':2, 'd':3}, inplace = True)
df2.head()

In [23]:
df2.value_counts("s70")

In [24]:
df2.describe()

In [25]:
df2.columns

In [26]:
df2["s52"].describe()

In [27]:
df2.value_counts("s52")

In [28]:
df2['s52'].replace({'l':1, 'o':0}, inplace = True)

In [29]:
df2.value_counts("s52")

In [30]:
df2.value_counts("s69")

In [31]:
df2['s69'].replace({'x':0, '~1':1,'C`':2, '0':3}, inplace = True)

In [32]:
df2.value_counts("s69")

In [33]:
df2.info()

# Changing datatypes from object to int

In [34]:
df2['s52'] = df2['s52'].astype('int64')

In [35]:
df2.info()

# Checking if the dataset is balanced or imbalanced

In [36]:
df2.value_counts("label")

# Balancing Imbalanced Dataset

In [37]:
from sklearn.utils import resample
#create two different dataframe of majority and minority class 
df_majority = df2[(df2['label']==0)] 
df_minority = df2[(df2['label']==1)] 
# upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,    # sample with replacement
                                 n_samples= 23562, # to match majority class
                                 random_state=42)  # reproducible results
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_minority_upsampled, df_majority])

In [38]:
df_upsampled.value_counts("label")

# Converting dataset into features:X and Label:Y

In [39]:
X = df_upsampled.drop(columns = ['id','label']) # X contains the features 
y= df_upsampled['label']
print(X.shape)
X.head
# df.drop(columns =['C', 'D'])

In [40]:
X.head

In [41]:
X.columns

In [42]:
X.info()

In [43]:
y.head

In [44]:
print(X.shape)
print(y.shape)

# Data Visualization

In [45]:
features = ['gender', 's11', 's12', 's13', 's16', 's17', 's18', 's48', 's52', 's58',
       's69', 's70', 's71', 'n3', 'n12', 'n13', 'n15']

for f in features:
    sns.countplot(x = f, data = df2, palette = 'Set3')
    plt.show()

In [46]:
sns.countplot(x = 'label', data = df2, hue = 'gender');

In [47]:
sns.kdeplot(x = "gender", data = df2);

# Using Pearson's Correlation Coefficient to find Relationships between variables

In [48]:
corr = X.corr()
mask1 = np.triu(np.ones_like(corr, dtype=bool))
mask2 = np.tril(np.ones_like(corr, dtype=bool))
plt.figure(figsize = (12,10))
sns.heatmap(corr, annot = True, cmap = "seismic", mask = mask1);

In [49]:
# with the following function we can select highly correlated features
# it will remove the first feature that is highly correlated with another feature

def correlation(dataset, threshold):
    col_corr = set() # Set of all the names of the redundant columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if(abs(corr_matrix.iloc[i, j])) > threshold:
                colname = corr_matrix.columns[i]
                col_corr.add(colname)
    return col_corr

In [50]:
corr_features = correlation(X, 0.9)
print(len(corr_features))
print(corr_features)

In [51]:
#Droping simillar features
X = X.drop(columns = corr_features)

print(X.shape)


In [52]:
X.columns

In [53]:
corr = X.corr() #feature selection pearsons coorelation
corr

plt.figure(figsize = (12,10))
sns.heatmap(corr, annot = True, cmap = "seismic");

In [54]:
corr_features= correlation(X,0.90)
print(len(corr_features))
print(corr_features)

# Train Test Split

In [55]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state =42)

In [56]:
Y_pre = df_upsampled['label']
target_names = list(set(Y_pre))
target_names

# Applying ML Models

In [57]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn import metrics

**SVM**

In [ ]:
from sklearn.svm import SVC
SVMclassifer = SVC(kernel='linear',probability=True)
SVMclassifer .fit(X_train, y_train)
y_pred = SVMclassifer.predict(X_test)
print('Test set accuracy %s' % accuracy_score(y_test, y_pred))



In [ ]:
y_pred_proba = SVMclassifer.predict_proba(X_test)[::,1]
auc = metrics.roc_auc_score(y_test, y_pred_proba)

#print AUC score
print(auc)

**XGBoost**

In [ ]:
from xgboost import XGBClassifier
XGclassifer = XGBClassifier()
XGclassifer.fit(X_train, y_train)
y_pred = XGclassifer.predict(X_test)
print('Test set accuracy %s' % accuracy_score(y_test, y_pred))


In [ ]:
y_pred_proba = XGclassifer.predict_proba(X_test)[::,1]
auc = metrics.roc_auc_score(y_test, y_pred_proba)

#print AUC score
print(auc)

**Random Forest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RFclassifer = RandomForestClassifier(n_estimators=100)
RFclassifer.fit(X_train, y_train)
y_pred = RFclassifer.predict(X_test)
print('Test set accuracy %s' % accuracy_score(y_test, y_pred))

In [ ]:
y_pred_proba = RFclassifer.predict_proba(X_test)[::,1]
auc = metrics.roc_auc_score(y_test, y_pred_proba)

#print AUC score
print(auc)

**Naive Bayes**

In [ ]:
from sklearn.naive_bayes import GaussianNB
NBclassifer = GaussianNB()
NBclassifer.fit(X_train, y_train)
y_pred = NBclassifer.predict(X_test)
print('Test set accuracy %s' % accuracy_score(y_test, y_pred))

In [ ]:
y_pred_proba = NBclassifer.predict_proba(X_test)[::,1]
auc = metrics.roc_auc_score(y_test, y_pred_proba)

#print AUC score
print(auc)

**AdaBoost**

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
ABclassifer = AdaBoostClassifier(n_estimators=60,
                         learning_rate=1)
ABclassifer.fit(X_train, y_train)
y_pred = ABclassifer.predict(X_test)
print('Test set accuracy %s' % accuracy_score(y_test, y_pred))

In [ ]:
y_pred_proba = ABclassifer.predict_proba(X_test)[::,1]
auc = metrics.roc_auc_score(y_test, y_pred_proba)

#print AUC score
print(auc)

**Logistics Regression**

In [ ]:
from sklearn.linear_model import LogisticRegression
LRclassifer = LogisticRegression()
LRclassifer.fit(X_train, y_train)
y_pred = LRclassifer.predict(X_test)
print('Test set accuracy %s' % accuracy_score(y_test, y_pred))

In [ ]:
y_pred_proba = LRclassifer.predict_proba(X_test)[::,1]
auc = metrics.roc_auc_score(y_test, y_pred_proba)

#print AUC score
print(auc)

# Using explainable to explain the output of the ML Models

# SHAP

In [ ]:
import shap

In [ ]:
# Create Tree Explainer object that can calculate shap values
explainer = shap.TreeExplainer(RFclassifer)

In [ ]:
df2.loc[[421]]

In [ ]:
# Calculate Shap values
choosen_instance = X_test.loc[[421]]
shap_values = explainer.shap_values(choosen_instance)
shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_values[1], choosen_instance)

In [ ]:
df2.loc[[55]]

In [ ]:
# Calculate Shap values
choosen_instance = X_test.loc[[55]]
shap_values = explainer.shap_values(choosen_instance)
shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_values[1], choosen_instance)

In [ ]:
shap.summary_plot(shap_values, X_train)

# LIME

In [ ]:
import lime
import lime.lime_tabular

In [ ]:
#Creating Lime Object
predict_fn_rf = lambda x: RFclassifer.predict_proba(x).astype(float)
X = X_train.values
explainer = lime.lime_tabular.LimeTabularExplainer(X,feature_names = X_train.columns,class_names=['0','1'],kernel_width=5)

In [ ]:
df2.loc[[421]]

In [ ]:
# Calculate Lime values
choosen_instance = X_test.loc[[421]].values[0]
exp = explainer.explain_instance(choosen_instance, predict_fn_rf,num_features=10)
exp.show_in_notebook(show_all=False)

In [ ]:
choosen_instance = X_test.loc[[55]].values[0]
exp = explainer.explain_instance(choosen_instance, predict_fn_rf,num_features=10)
exp.show_in_notebook(show_all=False)